In [9]:
import pandas as pd
import numpy as np
from datetime import timedelta
import datetime as dt
from datetime import date

In [10]:
data = pd.read_csv('us_time_series.csv')
data.head()

,Date,Confirmed,Deaths
0,2020-01-23,0.0,0.0
1,2020-01-24,1.0,0.0
2,2020-01-25,0.0,0.0
3,2020-01-26,3.0,0.0
4,2020-01-27,0.0,0.0


In [11]:
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
df = pd.read_csv(url)

In [12]:
df.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1398,1413,1420,1432,1442,1447,1463,1619,1624,1664
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,4639,4678,4722,4752,4781,4800,4812,5003,5021,5033
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,618,618,618,620,622,626,629,809,809,824
3,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,...,564,566,574,576,578,581,580,612,617,619
4,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,...,1082,1099,1109,1114,1121,1128,1139,1487,1504,1527


In [13]:
#Drop uncessary columns
df = df.drop(['UID','iso2',"iso3",'code3','Country_Region','Combined_Key'],axis=1)

#Extract variable with dates
dates = df.columns[6:]

#Melt the dataset from row into columns
us_confirmed = pd.melt(df,id_vars = ['Admin2','Province_State','FIPS','Lat','Long_'],value_vars=dates, var_name='Date', value_name='Confirmed')


In [14]:
us_confirmed.head()

,Admin2,Province_State,FIPS,Lat,Long_,Date,Confirmed
0,Autauga,Alabama,1001.0,32.539527,-86.644082,1/23/20,0
1,Baldwin,Alabama,1003.0,30.727750,-87.722071,1/23/20,0
2,Barbour,Alabama,1005.0,31.868263,-85.387129,1/23/20,0
3,Bibb,Alabama,1007.0,32.996421,-87.125115,1/23/20,0
4,Blount,Alabama,1009.0,33.982109,-86.567906,1/23/20,0


In [15]:
us_confirmed['Date'] = pd.to_datetime(us_confirmed['Date'])
us_confirmed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 798260 entries, 0 to 798259
Data columns (total 7 columns):
Admin2            796826 non-null object
Province_State    798260 non-null object
FIPS              795870 non-null float64
Lat               798260 non-null float64
Long_             798260 non-null float64
Date              798260 non-null datetime64[ns]
Confirmed         798260 non-null int64
dtypes: datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 42.6+ MB


In [27]:
us_confirmed = us_confirmed.set_index('Date')

In [38]:
us_confirmed = us_confirmed.reset_index()
df = us_confirmed[['Date','Province_State','Admin2','Confirmed']]

In [29]:
yesterday = (date.today() - timedelta(days = 1))
days_14 = (yesterday - timedelta(days = 14)).strftime('%#m/%#d/%y')
yesterday = yesterday.strftime('%#m/%#d/%y')
# days_14 = days_14.strftime("%Y-%m-%d")

In [130]:
df = df.set_index('Date')

In [131]:
days_14 = df.loc[days_14:yesterday]

In [163]:
days_14.sort_values(by = ['Province_State','Admin2'])
state_county = days_14[['Province_State','Admin2']]
state_county = state_county.drop_duplicates()
state_county = state_county.reset_index()
state = []
county = []
average  = []
for i in range(0,len(state_county)):
    temp_state = state_county.loc[i,'Province_State']
    temp_county = state_county.loc[i,'Admin2']
    mean_average = days_14[(days_14['Province_State'] == temp_state) & (days_14['Admin2']==temp_county)]['Confirmed'].diff().mean()
    state.append(temp_state)
    county.append(temp_county)
    average.append(mean_average)

In [166]:
average_df = pd.DataFrame({'state':state,'county':county,'average':average})
average_df.head()

,state,county,average
0,Alabama,Autauga,19.642857
1,Alabama,Baldwin,37.571429
2,Alabama,Barbour,13.928571
3,Alabama,Bibb,5.357143
4,Alabama,Blount,33.357143


In [171]:
average_df.to_csv('us_moving_avg.csv',index=False)